In [0]:
# Mount code
from google.colab import drive
drive.mount('/content/gdrive')

from keras.models import load_model
import keras.metrics
import keras.preprocessing.image as image
import numpy as np

def top_3_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)


def top_2_accuracy(y_true, y_pred):
    return keras.metrics.top_k_categorical_accuracy(y_true, y_pred, k=2)


keras.metrics.top_3_accuracy = top_3_accuracy
keras.metrics.top_2_accuracy = top_2_accuracy

model = load_model('/content/gdrive/My Drive/Colab/model.h5') # change to your model.h5 file path

In [0]:
import keras
from PIL import Image
from keras.preprocessing.image import ImageDataGenerator

generator = ImageDataGenerator(
    preprocessing_function=
    keras.applications.mobilenet.preprocess_input,
    rotation_range=180,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

generator_with_aug = generator.flow_from_directory(
    '/content/gdrive/My Drive/microservices-hospital-webapp/skin-cancer-model-training/divided_by_classes',
    target_size=(224, 224),
    batch_size=1) #There is only 1 picture in the directory


In [0]:
activations = model.predict_generator(generator_with_aug, steps=20)

In [0]:
%matplotlib inline
import matplotlib
from keras.models import Model
import matplotlib.pyplot as plt

img_path = '/content/gdrive/My Drive/microservices-hospital-webapp/skin-cancer-model-training/divided_by_classes/mel/ISIC_0024310.jpg'
img = image.load_img(img_path, target_size=(224, 224))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)
img_tensor /= 224.
plt.imshow(img_tensor[0])
plt.show()
print(img_tensor.shape)


In [0]:
layer_outputs = [layer.output for layer in model.layers[1:]]
activation_model = Model(inputs=model.input, outputs=layer_outputs)

activations = activation_model.predict(img_tensor)

In [0]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)
plt.matshow(first_layer_activation[0, :, :, 2], cmap='viridis')

In [0]:
model.summary()

In [0]:
layer_names = []

for layer in model.layers[1:-3]:
    layer_names.append(layer.name) # Names of the layers, so you can have them as part of your plot 

images_per_row = 16


for layer_name, layer_activation in zip(layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).

    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    
    display_grid = np.zeros((size * n_cols, images_per_row * size))
    # print('heree')
    for col in range(n_cols): # Tiles each filter into a big horizontal grid
        for row in range(images_per_row):
              channel_image = layer_activation[0,
                                               :, :,
                                               col * images_per_row + row]
              channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
              channel_image /= channel_image.std()
              channel_image *= 64
              channel_image += 128
              channel_image = np.clip(channel_image, 0, 255).astype('uint8')
              display_grid[col * size : (col + 1) * size, # Displays the grid
                         row * size : (row + 1) * size] = channel_image
             
            
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')

In [0]:
!git clone https://github.com/Joro97/microservices-hospital-webapp
  
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
# Put the file id for the input zip from your google drive
# This is TsvetanKonstantinov input.zip file id. You will not be able to use it
file_id = '13d_Y4BYuIYIHVJvUE95c_hFTZZ0c8_tK'
downloaded = drive.CreateFile({'id': file_id})
print(downloaded)

# The python scripts expect the input.zip to be in this directory
downloaded.GetContentFile("microservices-hospital-webapp/skin-cancer-model-training/input.zip")

In [0]:
from google.colab import files
files.upload()

!mv definitions.py /content/microservices-hospital-webapp/skin-cancer-model-training/src/train/definitions.py

In [0]:
import os

ROOT_DIR = '/content/microservices-hospital-webapp/skin-cancer-model-training'

INPUT_PATH = os.path.join(ROOT_DIR, 'For Drive')

INPUT_ZIP_PATH = os.path.join(ROOT_DIR, 'input.zip')
VALIDATION_FOLDER_PATH = os.path.join(INPUT_PATH, 'validation')

In [0]:
import zipfile

with zipfile.ZipFile(INPUT_ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(ROOT_DIR)

In [0]:
LABELS = [
    'nv',
    'mel',
    'bkl',
    'bcc',
    'akiec',
    'vasc',
    'df'
]

In [0]:
def get_num_of_images(path):
    num = 0

    for label in LABELS:
        imgs = os.listdir(os.path.join(path, label))
        num += len(imgs)

    print('There are {} images in {}'.format(path, num))

    return num

In [0]:
import numpy as np

num_val_samples = get_num_of_images(VALIDATION_FOLDER_PATH)

train_batch_size = 10
val_batch_size = 10
image_size = 224

datagen = ImageDataGenerator(preprocessing_function=keras.applications.mobilenet.preprocess_input)

valid_batches = datagen.flow_from_directory(VALIDATION_FOLDER_PATH,
                                            target_size=(image_size,image_size),
                                            batch_size=val_batch_size)

val_steps = np.ceil(num_val_samples / val_batch_size)

In [0]:
model.metrics_names

In [0]:
val_loss, val_cat_acc, val_top_2_acc, val_top_3_acc = model.evaluate_generator(valid_batches,
                                                                               steps=val_steps)

In [0]:
print('val_loss:', val_loss)
print('val_cat_acc:', val_cat_acc)
print('val_top_2_acc:', val_top_2_acc)
print('val_top_3_acc:', val_top_3_acc)

In [0]:
val_labels = valid_batches.classes
print(valid_batches.class_indices)

In [0]:
predictions = model.predict_generator(valid_batches,
                                      steps=val_steps)

In [0]:
predictions.shape

In [0]:
val_labels.shape

In [0]:
predictions.argmax(1)

In [0]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [0]:
cm = confusion_matrix(val_labels, predictions.argmax(axis=1))

In [0]:
cm_plot_labels = ['akiec', 'bcc', 'bkl', 'df', 'mel','nv', 'vasc']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [0]:
# Get the index of the class with the highest probability score
y_pred = np.argmax(predictions, axis=1)

# Get the labels of the test images.
y_true = valid_batches.classes


In [0]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(y_true, y_pred, target_names=cm_plot_labels)

print(report)